---
# Section 1.4: Positive Definite Systems; Cholesky Decomposition
---

After triangular linear systems, the next easiest type of linear system to solve is the **symmetric positive definite** linear system.

A matrix $A \in \mathbb{R}^{n \times n}$ is **symmetric** if $A = A^T$.

For example,

$$
A = 
\begin{bmatrix}
1 & 2 & 3 \\
2 & 4 & 5 \\
3 & 5 & 6
\end{bmatrix}
$$

is a symmetric matrix.

A matrix $A \in \mathbb{R}^{n \times n}$ is called **positive definite** if $A$ is symmetric and 

$$
x^T A x > 0, \qquad \text{for all nonzero $x \in \mathbb{R}^n$.}
$$

---

### Exercise:

Let $A = \begin{bmatrix} 2 & -1 \\ -1 & 2 \end{bmatrix}$ and $x = \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \neq 0$. Prove that $x^T A x > 0$.

---

## Properties of positive definite matrices

(1) If $A$ is positive definite, then $A$ is nonsingular.

(2) If $A$ is positive definite, then $Ax = b$ has a unique solution.

(3) If $A = M^TM$ for some nonsingular $M \in \mathbb{R}^{n \times n}$, then $A$ is positive definite.

(4) $A$ is positive definite $\Longleftrightarrow$ all eigenvalues of $A$ are positive.

---
### Exercise:

Prove property (1) and deduce property (2).

---
### Exercise:

Based on property (3), write a `Julia` function `A = randspd(n)` that randomly generates an $n \times n$ symmetric positive definite matrix $A$. Check that the matrix $A$ is positive definite by using property (4).

---

## The Cholesky Decomposition

It turns out that every positive definite matrix $A$ can be written as $A = M^TM$, for some nonsingular matrix $M$.

> ### Cholesky Decomposition Theorem
> Let $A$ be positive definite.
> Then there is a **unique upper-triangular matrix** $R$ with **positive diagonal entries** such that
> $$A = R^T R.$$

We will see a proof of this theorem later.

---

### Exercise:

Computing the Cholesky factor of a random positive definite matrix $A$ using the `Julia` function `cholesky`. Check that the output is correct.

---

### Exercise:

The function `cholesky` will detect if $A$ is not positive definite. In fact, this is the **most efficient** method for testing if a symmetric matrix is positive definite or not.

See what happens when you give the `cholesky` function a matrix that is symmetric but not positive definite.

---

### Exercise:

Let $R$ be the Cholesky factor of a positive definite matrix $A$. 

1. Propose an algorithm for solving $Ax = b$ using the Cholesky factorization of $A$. Your algorithm should have **two** steps, a forward-substitution and a backward-substitution.

2. What is the flop count of your algorithm?

3. Implement your algorithm in `Julia`. Check that the output is correct.

---
### Exercise:

Compute the Cholesky factor of $A$ by hand by writing $A = R^T R$ as follows.

$$ 
\begin{bmatrix}
 4 & -2 &  4 \\
-2 & 10 & -2 \\
 4 & -2 &  8 \\
\end{bmatrix} =
\begin{bmatrix}
r_{11} \\
r_{12} & r_{22} \\
r_{13} & r_{23} & r_{33} \\
\end{bmatrix}
\begin{bmatrix}
r_{11} & r_{12} & r_{13} \\
       & r_{22} & r_{23} \\
       &        & r_{33} \\
\end{bmatrix}
$$

---

### General Inner-Product Cholesky Formula

The general formulas for the entries $r_{ij}$ of the Cholesky factor $R$ of a positive definite matrix $A$ are given by

$$
\begin{align}
r_{ii} &= \sqrt{a_{ii} - \sum_{k=1}^{i-1} r_{ki}^2}, \\
r_{ij} &= \left(a_{ij} - \sum_{k=1}^{i-1} r_{ki} r_{kj}\right)\bigg/r_{ii}, \qquad j = i+1, \ldots, n. \\
\end{align}
$$

---

### Exercise:

Complete the following `Julia` function for the inner-product version of Cholesky's Algorithm.

In [9]:
"""
 CHOL_IP   Inner-product form of Cholesky's algorithm
    R = CHOL_IP(A) computes the Cholesky factor of A using the
    inner-product version of Cholesky's algorithm.
"""
function chol_ip(A::Matrix{Float64})

    # Check that A is a square and symmetric matrix
    issymmetric(A) || error("Matrix must be symmetric.")
        
    # Initialize R to be the upper-triangular part of A
    R = triu(A)

    n = size(A, 1)

    for i = 1:n
        # R[i,i] = sqrt(A[i,i] - sum_{k=1}^{i-1} R[k,i]^2)
        
        # R[i,j] = (A[i,j] - sum_{k=1}^{i-1} R[k,i]*R[k,j]) / R[i,i]
        
    end
    
    return R
end


chol_ip

---

### Exercise:

Develop the **Outer-Product Form of Cholesky's Algorithm** by partitioning $A = R^TR$ as:

$$
\begin{bmatrix}
a_{11} & b^T \\
b & \hat{A} \\
\end{bmatrix} 
= 
\begin{bmatrix}
r_{11} & 0 \\
s & \hat{R}^T \\
\end{bmatrix}
\begin{bmatrix}
r_{11} & s^T \\
0 & \hat{R} \\
\end{bmatrix}.
$$

1. Use block-matrix multiplication to multiply the matrices $R^T$ and $R$.
2. Based on the equations from part (1), derive a recursive algorithm for computing $R$.

---

### Exercise:

Complete the following `Julia` function for the recursive version of Cholesky's Algorithm.

In [16]:
"""
 CHOL_ROP   Recursive outer-product form of Cholesky's algorithm
    R = CHOL_ROP(A) computes the Cholesky factor of A using the
    recursive outer-product version of Cholesky's algorithm.
"""
function chol_rop(A::Matrix{Float64})

    # Check that A is a square and symmetric matrix
    issymmetric(A) || error("Matrix must be symmetric.")

    n = size(A, 1)

    # Initialize R = 0 and the same size and datatype as A
    R = zeros(n, n)
    
    #######################
    # Your code goes here #
    #######################
    
    return R
    
end

chol_rop

---

### Exercise:

Develop the **Bordered Form of Cholesky's Algorithm** by partitioning $A = R^TR$ as:

$$
\begin{bmatrix}
\hat{A} & c \\
c^T & a_{nn} \\
\end{bmatrix} 
= 
\begin{bmatrix}
\hat{R}^T & 0 \\
h^T & r_{nn} \\
\end{bmatrix}
\begin{bmatrix}
\hat{R} & h \\
0 & r_{nn} \\
\end{bmatrix}
$$

1. Use block-matrix multiplication to multiply the matrices $R^T$ and $R$.
2. Based on the equations from part (1), derive a recursive algorithm for computing $R$.
3. Based on your recursive algorithm, determine the number of flops for computing the Cholesky decomposition of a positive definite matrix $A$.

---

## Proof of the Cholesky Decomposition Theorem

Recall:

> ### Cholesky Decomposition Theorem
> Let $A$ be positive definite.
> Then there is a **unique upper-triangular matrix** $R$ with **positive diagonal entries** such that
> $$A = R^T R.$$

We will prove this theorem using mathematical induction. First we need to prove a couple of lemmas.

> ### Lemma 1
> Let $A$ be positive definite and partitioned as
$$
A = 
\begin{bmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{bmatrix}
$$
where $A_{11} \in \mathbb{R}^{n_1 \times n_1}$ and $A_{22} \in \mathbb{R}^{n_2 \times n_2}$. Then $A_{11}$ and $A_{22}$ are positive definite.

> ### Lemma 2
> Let $A, X \in \mathbb{R}^{n \times n}$, with $A$ positive definite and $X$ nonsingular.
Then $B = X^TAX$ is positive definite.

---